In [189]:
import pandas as pd
import numpy as np

from numpy import exp, sqrt, dot
from scipy.spatial.distance import cdist

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import scale
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor

from hyperopt import STATUS_OK, hp, fmin, tpe, Trials, space_eval

### Import data

In [41]:
def load_data():
    full_data = pd.read_csv("Data/X.csv")
    train_y = pd.read_csv("Data/y_train.csv")
    # Rename columns to something more interpretable
    columns = (["reflectance_" + str(i) for i in range(7)]
               + ["solar_" + str(i) for i in range(5)] + ["id"])
    full_data.columns = columns
    
    # Move ID column to the beginning
    id_column = full_data["id"]
    full_data.drop(labels=["id"], axis=1, inplace = True)
    full_data.insert(0, "id", id_column)
    
    # Add the target value column to the training part
    # in full_data
    split = 98000
    y_id_dict = train_y.set_index("id")["y"].to_dict()
    full_data.loc[:(split-1), "y"] = full_data.loc[:(split-1), "id"].map(y_id_dict)
    
    # Split into training and testing data
    train, test = full_data[:split], full_data[split:]
    return (train, test)

train, test = load_data()

### Setting parameters

In [42]:
random_seed = 9
# Set folds for out-of-fold prediction
n_folds = 5

### Preprocessing

In [43]:
cols_excl = ["id", "y"]
cols_orig = [c for c in train.columns if c not in cols_excl]

# Standardise data
train[cols_orig] = scale(train[cols_orig])
test[cols_orig] = scale(test[cols_orig])

### Data generation for MATLAB

In [4]:
#full_data = full_data.fillna(0)

#cols_excl = ["id", "y"]
#cols_orig = [c for c in full_data.columns if c not in cols_excl]

# Standardise data for LR
#full_data[cols_orig] = scale(full_data[cols_orig])

# I did standardise the data
#full_data.to_csv("aerosol_full.csv", encoding="utf-8", index=False, header=False)

### Python mean embedding

In [190]:
class Kernel(object):
    """ Kernel class from Zoltan Szabo"""

    def __init__(self, par=None):
        """ Initialization.

        Parameters
        ----------
        par : dictionary, optional
              Name of the kernel and its parameters (default is
              {"name": "RBF", "sigma": 1}). The name of the kernel comes
              from "RBF", "exponential", "Cauchy", "student", "Matern3p2",
              "Matern5p2", "polynomial", "ratquadr" (rational quadratic),
              "invmquadr" (inverse multiquadr).
        """
        if par is None:
            par = {"name": "RBF", "sigma": 1}

        name = par["name"]
        self.name = name

        # other attributes:
        if name == "RBF" or name == "exponential" or name == "Cauchy":
            self.sigma = par["sigma"]
        elif name == "student":
            self.d = par["d"]
        elif name == "Matern3p2" or name == "Matern5p2":
            self.l = par["l"]
        elif name == "polynomial":
            self.c = par["c"]
            self.exponent = par["exponent"]
        elif name == "ratquadr" or name == "invmquadr":
            self.c = par["c"]
        else:
            raise Exception("kernel=?")

    def gram_matrix(self, y1, y2):
        """  Compute the Gram matrix = [k(y1[i,:], y2[j,:])]; i, j: running.

        Parameters
        ----------
        y1 : (number of samples1, dimension)-ndarray
             One row of y1 corresponds to one sample.
        y2 : (number of samples2, dimension)-ndarray
             One row of y2 corresponds to one sample.

        Returns
        -------
        g : ndarray.
            Gram matrix of y1 and y2.
        """

        if self.name == "RBF":
            sigma = self.sigma
            g = cdist(y1, y2)
            g = exp(-g ** 2 / (2 * sigma ** 2))
        elif self.name == "exponential":
            sigma = self.sigma
            g = cdist(y1, y2)
            g = exp(-g / (2 * sigma ** 2))
        elif self.name == "Cauchy":
            sigma = self.sigma
            g = cdist(y1, y2)
            g = 1 / (1 + g ** 2 / sigma ** 2)
        elif self.name == "student":
            d = self.d
            g = cdist(y1, y2)
            g = 1 / (1 + g ** d)
        elif self.name == "Matern3p2":
            l = self.l
            g = cdist(y1, y2) 
            g = (1 + sqrt(3) * g / l) * exp(-sqrt(3) * g / l)
        elif self.name == "Matern5p2":
            l = self.l
            g = cdist(y1, y2)
            g = (1 + sqrt(5) * g / l + 5 * g ** 2 / (3 * l ** 2)) * \
                exp(-sqrt(5) * g / l)
        elif self.name == "polynomial":
            c = self.c
            exponent = self.exponent
            g = (dot(y1, y2.T) + c) ** exponent
        elif self.name == "ratquadr":
            c = self.c
            g = cdist(y1, y2) ** 2
            g = 1 - g / (g + c)
        elif self.name == "invmquadr":
            c = self.c
            g = cdist(y1, y2)
            g = 1 / sqrt(g ** 2 + c ** 2)
        else:
            raise Exception("kernel=?")

        return g

In [ ]:
def symmetrise(A):
    return(A + A.T - np.diag(A.diagonal()))

# Compute the linear kernel product of the mean embedding 
# of X1 and X2
def mean_embedding(X1, X2, kernel):
    k = Kernel(kernel)
    gram_mat = k.gram_matrix(X1, X2)
    # Number of instances in the bag
    N = float(gram_mat.shape[0])
    mu_X1_X2 = gram_mat.ravel().sum() / N**2
    return (mu_X1_X2)

# More than 25 minutes to compute
def compute_gram(df, kernel, theta):
    nb_bag = df["id"].nunique()
    K_matrix = np.zeros((nb_bag, nb_bag))
    print("Computing {0} Gram matrix for theta={1}:".format(kernel, theta))
    for i in range(nb_bag):
        if (i%50 == 0):
            print("Bag number: {0}". format(i))
        
        for j in range(i+1):
            # Compute mean embedding
            X1 = df.loc[train["id"] == (i+1), cols_orig].values
            X2 = df.loc[train["id"] == (j+1), cols_orig].values

            K_matrix[i, j] = mean_embedding(X1, X2, {'name': kernel, 'sigma': theta})
            
    return symmetrise(K_matrix)
        
K_cauchy = compute_gram(train, "Cauchy", 2**4)

In [127]:
G_train_cauchy

,0,1,2,3,4,5,6,7,8,9,...,970,971,972,973,974,975,976,977,978,979
0,0.991154,0.955328,0.969529,0.973129,0.908920,0.886188,0.902115,0.971101,0.908059,0.878027,...,0.972241,0.927735,0.954305,0.843801,0.962973,0.922967,0.941377,0.924171,0.976959,0.940458
1,0.955328,0.994694,0.962074,0.943584,0.958195,0.889848,0.914993,0.932280,0.959820,0.889961,...,0.938020,0.950168,0.927305,0.853174,0.940251,0.934127,0.947320,0.892866,0.959238,0.885163
2,0.969529,0.962074,0.996145,0.966989,0.945786,0.903766,0.927357,0.942365,0.929251,0.898904,...,0.946472,0.933018,0.948638,0.863751,0.957597,0.935604,0.940549,0.928179,0.990887,0.890669
3,0.973129,0.943584,0.966989,0.979929,0.891627,0.852030,0.873517,0.975832,0.881278,0.842075,...,0.952955,0.898075,0.927536,0.806302,0.940919,0.898540,0.935617,0.900440,0.974549,0.932908
4,0.908920,0.958195,0.945786,0.891627,0.995676,0.936040,0.948124,0.855386,0.977856,0.941444,...,0.905598,0.969282,0.920321,0.917565,0.928897,0.963797,0.934863,0.925191,0.938168,0.835651
5,0.886188,0.889848,0.903766,0.852030,0.936040,0.960130,0.939654,0.824101,0.925268,0.954629,...,0.911080,0.951053,0.924238,0.942606,0.930581,0.939656,0.871768,0.932048,0.908084,0.818047
6,0.902115,0.914993,0.927357,0.873517,0.948124,0.939654,0.965021,0.838032,0.950094,0.963026,...,0.897594,0.936528,0.912866,0.932247,0.919966,0.915281,0.883521,0.913542,0.915810,0.809028
7,0.971101,0.932280,0.942365,0.975832,0.855386,0.824101,0.838032,0.993016,0.852189,0.806896,...,0.957698,0.879065,0.916256,0.770735,0.932238,0.874546,0.916482,0.872618,0.958389,0.946369
8,0.908059,0.959820,0.929251,0.881278,0.977856,0.925268,0.950094,0.852189,0.983049,0.942466,...,0.899163,0.959648,0.909830,0.911446,0.918235,0.936235,0.917106,0.898656,0.917874,0.824724
9,0.878027,0.889961,0.898904,0.842075,0.941444,0.954629,0.963026,0.806896,0.942466,0.976619,...,0.888975,0.940674,0.905340,0.953461,0.912494,0.917370,0.865355,0.918008,0.892406,0.794167


### First base predictors

In [44]:
# Class for kernel ridge regression
class RidgeRegression(object):
    def __init__(self, l2_reg):
        self.l2_reg = l2_reg

    def fit(self, G, y):
        # Train size
        n_train = G.shape[0]
        ridge_mat = G + (self.l2_reg * n_train) * np.identity(n_train)
        self.ridge_mat = ridge_mat
        # Shape of y_train is (1, n_train)
        self.y_train = y

    def predict(self, G_test):
        y_test_hat = self.y_train.dot(np.linalg.solve(self.ridge_mat, G_test))
        return y_test_hat

In [45]:
# G_train and G_test are pandas dataframes
# krr is a kernel ridge regression
def oof_prediction(krr, G_train, y_train, G_test, n_folds, random_seed):
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=random_seed)
    n_train = G_train.shape[0]
    n_test = G_test.shape[1]
    oof_train = np.zeros(n_train)
    oof_test = np.zeros(n_test)
    oof_test_folds = np.zeros((n_test, n_folds))

    for i, (train_index, test_index) in enumerate(kf.split(G_train)):
        G_tr = G_train.loc[train_index, train_index].values
        y_tr = y_train[train_index].reshape((1, -1))
        G_te = G_train.loc[train_index, test_index].values

        krr.fit(G_tr, y_tr)
        oof_train[test_index] = krr.predict(G_te)
        G_test_partial = G_test.loc[train_index, :]
        oof_test_folds[:, i] = krr.predict(G_test_partial.values)

    oof_test = oof_test_folds.mean(axis=1)
    return oof_train, oof_test

* __Cauchy__: theta: 16, lambda: 2**(-23)
* __Matern3/2__: theta: 512, lambda: 2**(-33)
* __Matern5/2__: theta: 64, lambda: 2**(-31)
* __Rational quadratic__: theta: 512, lambda: 2**(-26)

In [46]:
nb_bags_train = 980
# Create a vector with the unique values of y for each ID.
y_train = train.groupby("id")["y"].median().values

In [108]:
# Load Gram matrices
def load_gram(csv_file, nb_bags_train):
    # Import data
    G = pd.read_csv(csv_file, header=None)
    idx_train = nb_bags_train - 1
    idx_test = nb_bags_train
    G_train = G.loc[:idx_train, :idx_train]
    G_test = G.loc[:idx_train, idx_test:]
    return (G_train, G_test)

In [109]:
# Define models and import Gram matrices
# Cauchy
l2_reg_cauchy = 2**(-23)
cauchy = RidgeRegression(l2_reg_cauchy)
G_train_cauchy, G_test_cauchy = load_gram("kernels_me/Cauchy_16.csv", nb_bags_train)

# Matern 3/2 -> removed as it gave inferior results
l2_reg_matern_32 = 2**(-33)
matern_32 = RidgeRegression(l2_reg_matern_32)
G_train_matern_32, G_test_matern_32 = load_gram("kernels_me/Matern_32_512.csv", nb_bags_train)

# Matern 5/2
l2_reg_matern_52 = 2**(-31)
matern_52 = RidgeRegression(l2_reg_matern_52)
G_train_matern_52, G_test_matern_52 = load_gram("kernels_me/Matern_52_64.csv", nb_bags_train)

# Rational quadratic
l2_reg_rquadr = 2**(-26)
rquadr = RidgeRegression(l2_reg_rquadr)
G_train_rquadr, G_test_rquadr = load_gram("kernels_me/rquadr_512.csv", nb_bags_train)

In [111]:
# Root mean squared error metric
def RMSE(y, y_hat):
    out = np.sqrt(mean_squared_error(y.reshape((-1,)), y_hat.reshape((-1,))))
    return (out)

In [115]:
# Create OOF train and test predictions
# Cauchy
cauchy_oof_train, cauchy_oof_test = oof_prediction(cauchy, G_train_cauchy, 
                                                   y_train, G_test_cauchy,
                                                   n_folds, random_seed)
#Matern 3/2
matern_32_oof_train, matern_32_oof_test = oof_prediction(matern_32, G_train_matern_32, 
                                                         y_train, G_test_matern_32,
                                                         n_folds, random_seed)
# Matern 5/2
matern_52_oof_train, matern_52_oof_test = oof_prediction(matern_52, G_train_matern_52, 
                                                         y_train, G_test_matern_52,
                                                         n_folds, random_seed)
# Rational quadratic
rquadr_oof_train, rquadr_oof_test = oof_prediction(rquadr, G_train_rquadr, 
                                                   y_train, G_test_rquadr,
                                                   n_folds, random_seed)
print("Training is finished.")

Training is finished.


In [116]:
RMSE(cauchy_oof_train, y_train)

0.67899005155348324

In [117]:
RMSE(matern_32_oof_train, y_train)

0.69624680290255281

In [118]:
RMSE(matern_52_oof_train, y_train)

0.68417462783849581

In [119]:
RMSE(rquadr_oof_train, y_train)

0.67813589585443934

### Second level prediction with DNN

In [120]:
kernel_train = pd.DataFrame({'cauchy': cauchy_oof_train,
                             'matern_32': matern_32_oof_train,
                             'matern_52': matern_52_oof_train,
                             'rquadr': rquadr_oof_train})

kernel_train["y"] = y_train

kernel_test = pd.DataFrame({'cauchy': cauchy_oof_test,
                            'matern_32': matern_32_oof_test,
                            'matern_52': matern_52_oof_test,
                            'rquadr': rquadr_oof_test})

cols_excl_kernel = ["y"]
cols_kernel = [c for c in kernel_train.columns if c not in cols_excl_kernel]

kernel_train.head()

,cauchy,matern_32,matern_52,rquadr,y
0,-3.682060,-3.609573,-3.602315,-3.658850,-3.998082
1,-4.281041,-4.398755,-4.306951,-4.295394,-4.137141
2,-2.664701,-2.792940,-2.704900,-2.647880,-2.694732
3,-3.664187,-3.659535,-3.720432,-3.657264,-3.296275
4,-3.749225,-3.813457,-3.762714,-3.722395,-3.181391


### Tuning neural network's parameters

In [62]:
def scoring_function(parameters):
    print("Training the model with parameters: ")
    print(parameters)
    average_RMSE = 0.0
    n_splits = 5
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=15)
    nb_fold = 0
    for train_index, validation_index in kf.split(kernel_train):
        nb_fold += 1
        train_fold, validation_fold = kernel_train.loc[train_index], kernel_train.loc[validation_index]

        #MPL Regressor
        model_dnn = MLPRegressor(hidden_layer_sizes=(parameters["nb_neurons"],),
                                 max_iter=parameters["steps"],
                                 alpha=parameters["MLP_l2_reg"],
                                 early_stopping=True,
                                 random_state=random_seed)
        model_dnn.fit(train_fold[cols_kernel], train_fold["y"])
        y_hat_train = model_dnn.predict(train_fold[cols_kernel])

        RMSE_train = RMSE(y_hat_train, train_fold["y"].values)
        print("Training RMSE: {0}".format(RMSE_train))

        y_hat_test = model_dnn.predict(validation_fold[cols_kernel])

        RMSE_test = RMSE(y_hat_test, validation_fold["y"].values)
        
        average_RMSE += RMSE_test
        print("Validation fold {0} RMSE: {1}".format(nb_fold, RMSE_test))
        print(model_dnn.n_iter_)

    average_RMSE /= n_splits

    print("Cross-validation score: {0}\n".format(average_RMSE))
    
    return {"loss": average_RMSE, "status": STATUS_OK}

In [63]:
# Grid to pick parameters from.
parameters_grid = {"steps"     : hp.choice("steps", np.arange(500, 600, 100, dtype=int)),
                   "nb_neurons": hp.choice("nb_neurons", np.arange(9, 10, 1, dtype=int)),
                   "MLP_l2_reg": hp.choice("MLP_l2_reg", np.power(2.0, np.arange(-9, -1)))
                  }
# Record the information about the cross-validation.
trials = Trials()

best = fmin(scoring_function, parameters_grid, algo=tpe.suggest, max_evals=4, 
            trials=trials)

Training the model with parameters: 
{'MLP_l2_reg': 0.015625, 'steps': 500, 'nb_neurons': 9}
Training RMSE: 0.684718260041
Validation fold 1 RMSE: 0.675408219075
131
Training RMSE: 0.676006751297
Validation fold 2 RMSE: 0.706441987051
138
Training RMSE: 0.723449725065
Validation fold 3 RMSE: 0.75997381004
90
Training RMSE: 0.685595306757
Validation fold 4 RMSE: 0.674064150788
128
Training RMSE: 0.696162943076
Validation fold 5 RMSE: 0.634931866164
134
Cross-validation score: 0.690164006624

Training the model with parameters: 
{'MLP_l2_reg': 0.015625, 'steps': 500, 'nb_neurons': 9}
Training RMSE: 0.684718260041
Validation fold 1 RMSE: 0.675408219075
131
Training RMSE: 0.676006751297
Validation fold 2 RMSE: 0.706441987051
138
Training RMSE: 0.723449725065
Validation fold 3 RMSE: 0.75997381004
90
Training RMSE: 0.685595306757
Validation fold 4 RMSE: 0.674064150788
128
Training RMSE: 0.696162943076
Validation fold 5 RMSE: 0.634931866164
134
Cross-validation score: 0.690164006624

Training

In [61]:
min(trials.losses())

0.6880125993100149

In [32]:
# Save the best parameters as a csv.
best_parameters = pd.DataFrame({key: [value] for (key, value) in 
                                zip(space_eval(parameters_grid, best).keys(),
                                    space_eval(parameters_grid, best).values())})
# Add the corresponding score.
best_parameters["score"] = min(trials.losses())
best_parameters.to_csv("best_parameters_12.csv", encoding="utf-8", index=False)

best_parameters

,MLP_l2_reg,nb_neurons,steps,score
0,0.015625,9,500,0.694888


### Prediction

In [33]:
best_parameters = pd.read_csv("best_parameters_12.csv", encoding="utf-8")
best_parameters

,MLP_l2_reg,nb_neurons,steps,score
0,0.015625,9,500,0.694888


In [34]:
#MPL Regressor
model_dnn = MLPRegressor(hidden_layer_sizes=(best_parameters["nb_neurons"][0]),
                         max_iter=best_parameters["steps"][0],
                         alpha=best_parameters["MLP_l2_reg"][0],
                         early_stopping=True,
                         random_state=random_seed)
model_dnn.fit(kernel_train[cols_kernel], y_train)

MLPRegressor(activation='relu', alpha=0.015625, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=9, learning_rate='constant',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=8, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [35]:
# Training error
RMSE(model_dnn.predict(kernel_train[cols_kernel]), y_train)

0.69496031445585071

In [114]:
model_dnn.n_iter_

18

In [36]:
# Prediction
y_hat_test = model_dnn.predict(kernel_test[cols_kernel])

test_pred = test.groupby("id")[["y"]].mean().reset_index()
test_pred["y"] = y_hat_test
test_pred.columns = ["Id", "y"]

In [37]:
# Save as a .csv
test_pred.to_csv("Prediction_12.csv", index=False)

In [39]:
test_pred

,Id,y
0,981,-3.722921
1,982,-3.801966
2,983,-3.367951
3,984,-3.395750
4,985,-3.461897
5,986,-5.114391
6,987,-4.286113
7,988,-2.756088
8,989,-3.851343
9,990,-3.927609


* Prediction 9: RMSE-validation: 0.72545.
Optimal theta: 16, lambda: 1.1921e-07 Public LB: 0.67536 base_kp.^[2:4]
* Prediction 10: cross-val RMSE: 0.673803 Public LB: 0.66464